In [74]:
from keras.models import load_model
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle

In [2]:
test_df = pd.read_csv("../input/test.csv")
train_df = pd.read_csv("../input/diuresis_train.csv")

In [9]:
test_d = test_df.Diuresis.values
train_d = train_df.iloc[:, 1].values

In [50]:
def find_closest_record(record):
    record = np.broadcast_to(record, train_d.shape)    
    distances = np.abs(np.subtract(record, train_d))
    
    idx = np.argsort(distances)[0]
    
    return train_df.iloc[idx, :].values[0]

In [56]:
mod_test = []
for i in tqdm(range(test_df.shape[0]), total = test_df.shape[0]):
    value = test_df.Diuresis.iloc[i]
    closest_id = find_closest_record(value)
    
    mod_record = train_df[train_df.people_ID == closest_id]
    mod_test.append(mod_record.values)

100%|██████████| 14498/14498 [00:24<00:00, 588.67it/s]


In [62]:
mod_test = np.squeeze(np.array(mod_test))[:, 2:]

In [75]:
f = open("../output/ss.pkl", "rb")
ss = pickle.load(f)
f.close()

In [76]:
x_test = ss.transform(mod_test)

In [77]:
x_test = x_test[..., None]

In [78]:
model = load_model("../models/lstm.h5")

In [79]:
y_pred = model.predict(x_test, batch_size = 256)

In [86]:
test_df["Diuresis"] = y_pred

In [90]:
test_df.to_csv("../input/task2_test.csv", index = False)